In [1]:
import pandas as pd

In [2]:
# Função para ler o arquivo CSV e retornar o dataframe
def ler_arquivo_csv(file_path):
    dados = pd.read_csv(file_path)
    dados.columns = dados.columns.str.strip()  # Remove espaços extras nos nomes das colunas
    return dados

# Função para encontrar os dados com a latitude e longitude correspondentes
def encontrar_dados_por_coordenadas(dados, lat, lon):
    linha = dados[(dados['latitude'] == lat) & (dados['longitude'] == lon)]
    if linha.empty:
        return None
    return linha.iloc[0]

# Função para calcular a probabilidade de queimada
def calcular_probabilidade(dados_linha):
    contagem_verdadeiras = 0

    if dados_linha['temperatura'] > 35:
        contagem_verdadeiras += 1
    if dados_linha['umidade'] > 30:
        contagem_verdadeiras += 1
    if dados_linha['pressao'] > 1002:
        contagem_verdadeiras += 1

    if contagem_verdadeiras == 3:
        return 100
    elif contagem_verdadeiras == 2:
        return 75
    elif contagem_verdadeiras == 1:
        return 50
    else:
        return 0

In [3]:
# Função principal para previsão de queimada com base em uma latitude e longitude
def prever_queimada(dados_queimadas, latitude, longitude):
    dados_linha = encontrar_dados_por_coordenadas(dados_queimadas, latitude, longitude)
    
    if dados_linha is None:
        return None  # Se não encontrar a linha, retorna None
    
    probabilidade = calcular_probabilidade(dados_linha)
    return probabilidade

In [4]:
# Função para iterar sobre o arquivo de latitudes e longitudes e gerar um novo CSV
def processar_multiplas_coordenadas(file_path_queimadas, file_path_coords, output_file):
    dados_queimadas = ler_arquivo_csv(file_path_queimadas)
    coords = ler_arquivo_csv(file_path_coords)

    resultados = []
    
    # Itera sobre cada linha de latitude e longitude no arquivo de coordenadas
    for _, linha in coords.iterrows():
        latitude = linha['latitude']
        longitude = linha['longitude']
        probabilidade = prever_queimada(dados_queimadas, latitude, longitude)
        
        if probabilidade is not None:
            resultados.append({'latitude': latitude, 'longitude': longitude, 'probabilidade': probabilidade})

    # Cria um novo DataFrame com os resultados e salva no arquivo CSV
    df_resultados = pd.DataFrame(resultados)
    df_resultados.to_csv(output_file, index=False)  # Salva o DataFrame no arquivo CSV

In [5]:
# Exemplo de uso
file_path_queimadas = './dadosParaArvore.csv'
file_path_coords = './modisFinal_2024.csv'
output_file = 'resultado_probabilidade.csv'  # Arquivo de saída

In [ ]:
# Processa as coordenadas e cria o novo arquivo com as probabilidades
processar_multiplas_coordenadas(file_path_queimadas, file_path_coords, output_file)